# 🧼 Image Demoising bằng Autoencoder (PyTorch)



## Quy trình
- Đọc dữ liệu ảnh theo chuẩn `ImageFolder`
- Thêm nhiễu: **Gaussian (normal)** / **Bernoulli (mask/dropout)** / **Poisson**
- Huấn luyện Autoencoder để **khử nhiễu**
- Đánh giá bằng **PSNR** và **MSE**
- Lưu ảnh minh hoạ: `*_noisy.png`, `*_denoised.png`, `*_clean.png`
- Lưu checkpoint tốt nhất theo PSNR: `best_autoencoder.pt`




In [1]:
import os
from pathlib import Path
from typing import Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, utils as vutils
import matplotlib.pyplot as plt

print("✅ Torch version:", torch.__version__)
print("✅ CUDA available:", torch.cuda.is_available())

ModuleNotFoundError: No module named 'numpy'

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import os
import time

# Detect Kaggle environment
IN_KAGGLE = os.path.exists('/kaggle')
print(f"🔍 Kaggle environment detected: {IN_KAGGLE}")

if IN_KAGGLE:
    print("✅ Running on Kaggle Notebook")
else:
    print("💻 Running on local machine")

# ==================== KIỂM TRA GPU ====================
print("\n" + "="*60)
print("🖥️ KIỂM TRA GPU")
print("="*60)
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Device Count: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)} - {torch.cuda.get_device_properties(i).total_memory / 1e9:.2f} GB")
    print(f"cudnn Version: {torch.backends.cudnn.version()}")
    print(f"cudnn Enabled: {torch.backends.cudnn.enabled}")
else:
    print("⚠️ GPU NOT AVAILABLE - Will use CPU (slower!)")
print("="*60 + "\n")

# ================== CẤU HÌNH CHO KAGGLE + GPU ====================

# 1) Đường dẫn dữ liệu - Tự động điều chỉnh cho Kaggle
if IN_KAGGLE:
    # Trên Kaggle: upload ảnh vào Kaggle dataset hoặc sử dụng working directory
    # Option 1: Nếu upload dataset vào Kaggle, dùng path này:
    # thu_muc_du_lieu = "/kaggle/input/your-dataset-name/thumbnails"
    
    # Option 2: Tạo dữ liệu từ working directory (recommended cho testing)
    thu_muc_du_lieu = "/kaggle/input/thumbnails"
    thu_muc_ket_qua = "/kaggle/working/outputs_denoise"
    
    # Để lưu và load checkpoint
    duong_dan_checkpoint_ae = "/kaggle/working/best_ae_model.pth"
    duong_dan_checkpoint_gan_g = "/kaggle/working/best_gan_generator.pth"
    duong_dan_checkpoint_gan_d = "/kaggle/working/best_gan_discriminator.pth"
    
else:
    # Trên máy local
    thu_muc_du_lieu = "./thumbnails"
    thu_muc_ket_qua = "./outputs_denoise"
    duong_dan_checkpoint_ae = "./best_ae_model.pth"
    duong_dan_checkpoint_gan_g = "./best_gan_generator.pth"
    duong_dan_checkpoint_gan_d = "./best_gan_discriminator.pth"

# 2) TỐI ƯU HÓA GPU - MULTI-GPU SUPPORT
use_cuda = torch.cuda.is_available()
n_gpus = torch.cuda.device_count() if use_cuda else 0

# Batch size tối ưu cho GPU - TĂNG GẤP ĐÔI NẾU CÓ 2 GPU
if use_cuda:
    # GPU có đủ VRAM, dùng batch size lớn hơn
    base_batch_size = 32 if "P100" in torch.cuda.get_device_name(0) else 16
    batch_size = base_batch_size * n_gpus  # Nhân với số GPU
    so_worker = 4 * n_gpus  # Tăng workers theo số GPU
    pin_memory = True  # Transfer data nhanh hơn
    
    # Tối ưu cuDNN
    torch.backends.cudnn.benchmark = True  # Tìm fastest implementation
    torch.backends.cudnn.enabled = True
else:
    batch_size = 8  # CPU: batch size nhỏ
    so_worker = 0
    pin_memory = False

# 3) Tham số huấn luyện
so_epoch = 20
learning_rate = 1e-3

# 4) Kích thước ảnh và độ rộng mạng
kich_thuoc_anh = 128
so_kenh_co_so = 32

# 5) Nhiễu
loai_nhieu = "normal"     # "normal" | "bernoulli" | "poisson"
do_manh_nhieu = 0.05      # normal: sigma; bernoulli: tỉ lệ mask; poisson: mức noise

# 6) Kiểu mục tiêu
kieu_muc_tieu = "clean"   # "clean" = khử nhiễu | "noisy" = identity-noise

# 7) Loss
ten_loss = "mse"          # "mse" | "l1"

# 8) Thiết bị
bat_buoc_cpu = False
thiet_bi = torch.device("cuda" if use_cuda else "cpu")

# 9) MULTI-GPU CONFIG
use_multi_gpu = n_gpus > 1 and use_cuda

print("="*60)
print("⚙️ CẤU HÌNH TRAINING TỐI ƯU")
print("="*60)
print(f"📁 Data dir: {thu_muc_du_lieu}")
print(f"📁 Output dir: {thu_muc_ket_qua}")
print(f"🖥️ Device: {thiet_bi}")
print(f"🎮 Number of GPUs: {n_gpus}")
if use_multi_gpu:
    print(f"🚀 MULTI-GPU ENABLED: Sử dụng {n_gpus} GPUs đồng thời!")
    print(f"   GPUs: {[torch.cuda.get_device_name(i) for i in range(n_gpus)]}")
else:
    print(f"⚠️ Single GPU mode (chỉ dùng 1 GPU)")
print(f"📊 Batch size: {batch_size} ({base_batch_size if use_cuda else batch_size} per GPU)")
print(f"🔄 Num workers: {so_worker}")
print(f"📌 Pin memory: {pin_memory}")
print(f"🚀 cuDNN Benchmark: {torch.backends.cudnn.benchmark}")
print(f"⏱️ Epochs: {so_epoch}")
print("="*60 + "\n")

In [ ]:
# ================== THIẾT LẬP THƯ MỤC + THIẾT BỊ ==================
thu_muc_ket_qua = Path(thu_muc_ket_qua)
thu_muc_ket_qua.mkdir(parents=True, exist_ok=True)

# Thiết bị đã được set ở cell trước
print("=" * 60)
print("🔧 THIẾT LẬP MÔI TRƯỜNG TRAINING")
print("=" * 60)
print(f"🖥️ Thiết bị: {thiet_bi} {'✅ GPU' if use_cuda else '❌ CPU'}")
print(f"📂 Dữ liệu: {Path(thu_muc_du_lieu).resolve()}")
print(f"📁 Kết quả: {thu_muc_ket_qua.resolve()}")
print(f"🖼️ Kích thước: {kich_thuoc_anh}x{kich_thuoc_anh}")
print(f"🎯 Nhiễu: {loai_nhieu} (σ={do_manh_nhieu})")
print(f"📉 Loss: {ten_loss.upper()}")
print(f"⏱️ Epochs: {so_epoch} | Batch: {batch_size} | Workers: {so_worker}")

# Kiểm tra dữ liệu có tồn tại không
duong_dan_du_lieu = Path(thu_muc_du_lieu)
if not duong_dan_du_lieu.exists():
    print(f"\n⚠️ CẢNH BÁO: {duong_dan_du_lieu} không tồn tại!")
    print(f"📌 Bạn cần upload dataset hoặc tạo thư mục dữ liệu trước khi chạy")
    if IN_KAGGLE:
        print("💡 Trên Kaggle: Hãy upload dataset hoặc sử dụng 'Add data' tính năng")

# Bật Mixed Precision Training nếu sử dụng GPU (tiết kiệm RAM, nhanh hơn)
if use_cuda and torch.cuda.get_device_capability(0)[0] >= 7:  # Compute Capability >= 7 (V100, A100, etc)
    print("\n✅ Mixed Precision Training ENABLED (amp)")
    from torch.cuda.amp import autocast, GradScaler
    use_amp = True
    scaler = GradScaler()
else:
    print("\n❌ Mixed Precision Training DISABLED")
    use_amp = False
    scaler = None

print("=" * 60 + "\n")


## 2) Hàm đánh giá (PSNR, MSE) + hàm thêm nhiễu

In [ ]:
# ================== HÀM ĐÁNH GIÁ & THÊM NHIỄU ====================

def psnr_tensor(x: torch.Tensor, y: torch.Tensor, max_pixel: float = 1.0) -> torch.Tensor:
    """Tính PSNR giữa 2 tensor (giả sử giá trị trong [0,1])."""
    mse = torch.mean((x - y) ** 2)
    if mse == 0:
        return torch.tensor(float('inf'), device=x.device)
    psnr = 20 * torch.log10(torch.tensor(max_pixel, device=x.device) / torch.sqrt(mse))
    return psnr

def mse_tensor(x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
    """Tính MSE giữa 2 tensor."""
    return torch.mean((x - y) ** 2)

def them_nhieu(anh: torch.Tensor, loai: str = "normal", do_manh: float = 0.05) -> torch.Tensor:
    """Thêm nhiễu vào ảnh - TỐI ƯU GPU."""
    if loai == "normal":
        nhieu = torch.randn_like(anh) * do_manh
    elif loai == "bernoulli":
        nhieu = torch.bernoulli(torch.full_like(anh, do_manh)) - 0.5
    elif loai == "poisson":
        nhieu = (torch.poisson(anh * 255.0 * do_manh) - anh * 255.0 * do_manh) / (255.0 * do_manh)
    else:
        nhieu = torch.zeros_like(anh)
    return torch.clamp(anh + nhieu, 0, 1)

print("✅ Đã định nghĩa các hàm: psnr_tensor(), mse_tensor(), them_nhieu()")
print("✅ Tất cả hàm đã tối ưu để chạy trực tiếp trên GPU (tensor operations)")

## 3) Kiến trúc Autoencoder

In [ ]:
# ================== KIẾN TRÚC AUTOENCODER ====================

class Autoencoder(nn.Module):
    """Autoencoder để khử nhiễu ảnh."""
    def __init__(self, so_kenh_vao: int = 3, kenh_co_so: int = 32):
        super().__init__()
        
        # Encoder: Downsampling
        self.encoder = nn.Sequential(
            nn.Conv2d(so_kenh_vao, kenh_co_so, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(kenh_co_so),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(kenh_co_so, kenh_co_so*2, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(kenh_co_so*2),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(kenh_co_so*2, kenh_co_so*4, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(kenh_co_so*4),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(kenh_co_so*4, kenh_co_so*8, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(kenh_co_so*8),
            nn.ReLU(inplace=True),
        )
        
        # Decoder: Upsampling
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(kenh_co_so*8, kenh_co_so*4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(kenh_co_so*4),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(kenh_co_so*4, kenh_co_so*2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(kenh_co_so*2),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(kenh_co_so*2, kenh_co_so, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(kenh_co_so),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(kenh_co_so, so_kenh_vao, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid()  # Output: [0, 1]
        )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward pass: Encode -> Decode."""
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


# Khởi tạo mô hình Autoencoder
mo_hinh = Autoencoder(so_kenh_vao=3, kenh_co_so=so_kenh_co_so).to(thiet_bi)

# ============ WRAP VỚI DataParallel NẾU CÓ NHIỀU GPU ============
if use_multi_gpu:
    print(f"\n🚀 Wrapping Autoencoder với DataParallel ({n_gpus} GPUs)")
    mo_hinh = nn.DataParallel(mo_hinh)
    print(f"✅ Model sẽ chạy trên: {mo_hinh.device_ids}")

print("✅ Autoencoder Architecture:")
print(mo_hinh)
print(f"\n✅ Model moved to {thiet_bi}")
if use_multi_gpu:
    print(f"🎮 Multi-GPU Training ENABLED: {n_gpus} GPUs đồng thời!")

In [ ]:
# ================== CHUẨN BỊ DỮ LIỆU ==================

# Định nghĩa Transform
transform = transforms.Compose([
    transforms.Resize((kich_thuoc_anh, kich_thuoc_anh)),
    transforms.ToTensor(),
])

# Load dữ liệu từ ImageFolder
try:
    dataset_full = ImageFolder(root=thu_muc_du_lieu, transform=transform)
    print(f"✅ Đã load {len(dataset_full)} ảnh từ {thu_muc_du_lieu}")
except Exception as e:
    print(f"❌ Lỗi khi load dữ liệu: {e}")
    print(f"📌 Đảm bảo cấu trúc thư mục: {thu_muc_du_lieu}/classA/ và classB/")
    dataset_full = None

if dataset_full is not None:
    # Split train/val (80/20)
    n = len(dataset_full)
    n_train = int(0.8 * n)
    n_val = n - n_train
    
    indices = np.random.permutation(n)
    indices_train = indices[:n_train]
    indices_val = indices[n_train:]
    
    dataset_train = torch.utils.data.Subset(dataset_full, indices_train)
    dataset_val = torch.utils.data.Subset(dataset_full, indices_val)
    
    print(f"📊 Train: {len(dataset_train)} | Val: {len(dataset_val)}")
    
    # DataLoader - TỐI ƯU CHO GPU
    train_loader = DataLoader(
        dataset_train, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=so_worker,
        pin_memory=pin_memory,  # ⚡ GPU tối ưu
        persistent_workers=(so_worker > 0)  # Giữ workers sống để tái sử dụng
    )
    val_loader = DataLoader(
        dataset_val, 
        batch_size=batch_size, 
        shuffle=False, 
        num_workers=so_worker,
        pin_memory=pin_memory,
        persistent_workers=(so_worker > 0)
    )
    
    print(f"✅ DataLoader tạo thành công")
    print(f"💡 Mỗi epoch: {len(train_loader)} batches training | {len(val_loader)} batches validation")
else:
    train_loader = None
    val_loader = None
    print("⚠️ Không thể tạo DataLoader")


## 4) Hàm đánh giá và lưu ảnh minh hoạ

In [ ]:
# ================== HÀM ĐÁNH GIÁ & MINH HỌA ==================

def psnr_tensor(x: torch.Tensor, y: torch.Tensor, max_pixel: float = 1.0) -> torch.Tensor:
    """Tính PSNR giữa 2 tensor (giả sử giá trị trong [0,1])."""
    mse = torch.mean((x - y) ** 2)
    if mse == 0:
        return torch.tensor(float('inf'), device=x.device)
    psnr = 20 * torch.log10(torch.tensor(max_pixel, device=x.device) / torch.sqrt(mse))
    return psnr

def mse_tensor(x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
    """Tính MSE giữa 2 tensor."""
    return torch.mean((x - y) ** 2)

def them_nhieu(anh: torch.Tensor, loai: str = "normal", do_manh: float = 0.05) -> torch.Tensor:
    """Thêm nhiễu vào ảnh."""
    if loai == "normal":
        nhieu = torch.randn_like(anh) * do_manh
    elif loai == "bernoulli":
        nhieu = torch.bernoulli(torch.full_like(anh, do_manh)) - 0.5
    elif loai == "poisson":
        nhieu = (torch.poisson(anh * 255.0 * do_manh) - anh * 255.0 * do_manh) / (255.0 * do_manh)
    else:
        nhieu = torch.zeros_like(anh)
    return torch.clamp(anh + nhieu, 0, 1)

def danh_gia(mo_hinh: nn.Module, loader: DataLoader, device: torch.device, 
             loai_nhieu: str, do_manh_nhieu: float) -> tuple:
    """Đánh giá mô hình và trả về PSNR, MSE, loss."""
    mo_hinh.eval()
    psnr_vals, mse_vals, loss_vals = [], [], []
    loss_fn = nn.MSELoss()
    
    with torch.no_grad():
        for anh_sach, _ in loader:
            anh_sach = anh_sach.to(device)
            anh_nhieu = them_nhieu(anh_sach, loai_nhieu, do_manh_nhieu)
            
            dau_ra = mo_hinh(anh_nhieu)
            
            psnr_vals.append(psnr_tensor(dau_ra, anh_sach).item())
            mse_vals.append(mse_tensor(dau_ra, anh_sach).item())
            loss_vals.append(loss_fn(dau_ra, anh_sach).item())
    
    return np.mean(psnr_vals), np.mean(mse_vals), np.mean(loss_vals)

def luu_minh_hoa(mo_hinh: nn.Module, anh_mau: torch.Tensor, device: torch.device,
                loai_nhieu: str, do_manh_nhieu: float, thu_muc: Path, tag: str = ""):
    """Lưu ảnh minh hoạ denoising."""
    mo_hinh.eval()
    with torch.no_grad():
        anh_sach = anh_mau.to(device)
        anh_nhieu = them_nhieu(anh_sach, loai_nhieu, do_manh_nhieu)
        anh_khuc_phuc = mo_hinh(anh_nhieu)
    
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    
    for idx, (anh, tieu_de) in enumerate([
        (anh_nhieu[0], f"Nhiễu ({loai_nhieu})"),
        (anh_khuc_phuc[0], "Khôi phục"),
        (anh_sach[0], "Gốc")
    ]):
        axes[idx].imshow(anh.permute(1, 2, 0).cpu().numpy())
        axes[idx].set_title(tieu_de)
        axes[idx].axis('off')
    
    plt.tight_layout()
    ten_file = f"viz_{tag}.png" if tag else "viz.png"
    duong_dan_anh = thu_muc / ten_file
    plt.savefig(duong_dan_anh, dpi=100, bbox_inches='tight')
    plt.close()


## 5) Tải dữ liệu (ImageFolder)

In [ ]:
# ================== KIỂM TRA DATALOADER (ĐÃ TỐI ƯU GPU Ở CELL TRƯỚC) ====================

print("="*60)
print("✅ KIỂM TRA DATALOADER")
print("="*60)

if train_loader is not None and val_loader is not None:
    print(f"✅ Train DataLoader: {len(train_loader)} batches")
    print(f"✅ Val DataLoader: {len(val_loader)} batches")
    print(f"✅ Batch size: {batch_size}")
    print(f"✅ Num workers: {so_worker}")
    print(f"✅ Pin memory: {pin_memory} (GPU tối ưu)")
    print(f"✅ Non-blocking: True (GPU transfer tối ưu)")
    print("\n💡 DataLoader đã được tối ưu GPU ở cell trước - KHÔNG TẠO LẠI!")
else:
    print("❌ DataLoader chưa được khởi tạo! Chạy lại cell trước.")

print("="*60)

## 6) Huấn luyện mô hình

In [ ]:
# ================== HUẤN LUYỆN AUTOENCODER (TỐI ƯU GPU + MULTI-GPU) ==================

# Khởi tạo optimizer + loss
bo_toi_uu = optim.Adam(mo_hinh.parameters(), lr=learning_rate)

if ten_loss == "mse":
    ham_loss = nn.MSELoss()
elif ten_loss == "l1":
    ham_loss = nn.L1Loss()
else:
    raise ValueError("❌ ten_loss phải là 'mse' hoặc 'l1'")

best_psnr = -1.0
best_anh_mau = None
lich_su = {"loss_train": [], "psnr_val": [], "mse_val": []}

print("\n" + "="*60)
print("🚀 BẮT ĐẦU HUẤN LUYỆN AUTOENCODER")
if use_multi_gpu:
    print(f"🎮 Training với {n_gpus} GPUs: {[torch.cuda.get_device_name(i) for i in range(n_gpus)]}")
print("="*60)

time_start = time.time()

for epoch in range(1, so_epoch + 1):
    epoch_start = time.time()
    mo_hinh.train()
    tong_loss = 0.0
    
    with tqdm(train_loader, desc=f"Epoch {epoch}/{so_epoch}", leave=False) as pbar:
        for anh_sach, _ in pbar:
            anh_sach = anh_sach.to(thiet_bi, non_blocking=True)  # Non-blocking GPU transfer
            
            # Input: ảnh nhiễu
            anh_nhieu = them_nhieu(anh_sach, loai_nhieu, do_manh_nhieu).to(thiet_bi, non_blocking=True)
            
            # Target
            if kieu_muc_tieu == "clean":
                muc_tieu = anh_sach
            elif kieu_muc_tieu == "noisy":
                muc_tieu = them_nhieu(anh_sach, loai_nhieu, do_manh_nhieu).to(thiet_bi, non_blocking=True)
            else:
                raise ValueError("❌ kieu_muc_tieu phải là 'clean' hoặc 'noisy'")
            
            # Mixed precision forward pass
            if use_amp:
                with autocast():
                    anh_tai_tao = mo_hinh(anh_nhieu)
                    loss = ham_loss(anh_tai_tao, muc_tieu)
                
                # Backward with scaling
                bo_toi_uu.zero_grad()
                scaler.scale(loss).backward()
                scaler.unscale_(bo_toi_uu)
                torch.nn.utils.clip_grad_norm_(mo_hinh.parameters(), 1.0)
                scaler.step(bo_toi_uu)
                scaler.update()
            else:
                # Standard training - DataParallel tự động phân chia batch lên các GPU
                anh_tai_tao = mo_hinh(anh_nhieu)
                loss = ham_loss(anh_tai_tao, muc_tieu)
                
                bo_toi_uu.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(mo_hinh.parameters(), 1.0)
                bo_toi_uu.step()
            
            tong_loss += loss.item() * anh_sach.size(0)
            pbar.set_postfix({'loss': f'{loss.item():.6f}'})
    
    loss_train = tong_loss / len(train_loader.dataset)
    
    # Validate
    psnr_val, mse_val, _ = danh_gia(mo_hinh, val_loader, thiet_bi, loai_nhieu, do_manh_nhieu)
    
    lich_su["loss_train"].append(loss_train)
    lich_su["psnr_val"].append(psnr_val)
    lich_su["mse_val"].append(mse_val)
    
    epoch_time = time.time() - epoch_start
    
    print(f"[Epoch {epoch:02d}] ({epoch_time:.1f}s) "
          f"Loss: {loss_train:.6f} | "
          f"PSNR: {psnr_val:.2f}dB | "
          f"MSE: {mse_val:.6f}")
    
    # Chỉ lưu khi đạt PSNR tốt nhất
    if psnr_val > best_psnr:
        best_psnr = psnr_val
        best_anh_mau, _ = next(iter(val_loader))
        
        # Lưu model - nếu dùng DataParallel, lưu module.state_dict()
        if use_multi_gpu:
            torch.save(mo_hinh.module.state_dict(), duong_dan_checkpoint_ae)
        else:
            torch.save(mo_hinh.state_dict(), duong_dan_checkpoint_ae)
        
        print(f"  ✅ PSNR tốt nhất mới: {best_psnr:.4f}dB - Checkpoint lưu thành công!")

total_time = time.time() - time_start
print("\n" + "="*60)
print("✅ HOÀN THÀNH HỌC AUTOENCODER")
print("="*60)
print(f"Best PSNR: {best_psnr:.4f}")
print(f"Thời gian: {total_time/60:.1f} phút ({total_time/3600:.2f} giờ)")
print(f"Thiết bị: {thiet_bi} {'(GPU)' if use_cuda else '(CPU)'}")
if use_multi_gpu:
    print(f"🎮 Đã sử dụng {n_gpus} GPUs đồng thời!")
print("="*60)

# Lưu ảnh minh hoạ CHỈ TỪ MODEL TỐT NHẤT
if best_anh_mau is not None:
    print("\n" + "="*60)
    print("🖼️ LƯUẢNH MINH HỌA TỪ MODEL TỐT NHẤT")
    print("="*60)
    luu_minh_hoa(mo_hinh, best_anh_mau, thiet_bi, loai_nhieu, do_manh_nhieu, thu_muc_ket_qua, tag="best_autoencoder")
    print(f"✅ Đã lưu ảnh chất lượng cao nhất (PSNR={best_psnr:.4f}dB)")
    print("="*60)

# Clear cache
if use_cuda:
    torch.cuda.empty_cache()
    print("\n💾 GPU cache đã được xóa")

## 7) Vẽ biểu đồ lịch sử huấn luyện

In [ ]:
plt.figure()
plt.plot(lich_su["loss_train"])
plt.title("Loss huấn luyện (Train Loss)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

plt.figure()
plt.plot(lich_su["psnr_val"])
plt.title("PSNR validation (dB)")
plt.xlabel("Epoch")
plt.ylabel("PSNR (dB)")
plt.show()

plt.figure()
plt.plot(lich_su["mse_val"])
plt.title("MSE validation")
plt.xlabel("Epoch")
plt.ylabel("MSE")
plt.show()

## 8) (Tuỳ chọn) Load lại checkpoint tốt nhất và lưu ảnh minh hoạ

## 9) Image Denoising với GAN (Generator + Discriminator)

GAN Architecture:
- **Generator**: Mạng sinh ảnh sạch từ ảnh nhiễu
- **Discriminator**: Phân biệt ảnh sạch thực vs ảnh sinh ra
- **Loss**: Adversarial Loss + Reconstruction Loss (L1/MSE)


In [ ]:
# ==================== KIẾN TRÚC GAN ====================

class Generator(nn.Module):
    """Generator tương tự Autoencoder nhưng output activation khác."""
    def __init__(self, so_kenh_vao: int = 3, kenh_co_so: int = 32):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(so_kenh_vao, kenh_co_so, 3, stride=2, padding=1), nn.BatchNorm2d(kenh_co_so), nn.ReLU(True),
            nn.Conv2d(kenh_co_so, kenh_co_so*2, 3, stride=2, padding=1), nn.BatchNorm2d(kenh_co_so*2), nn.ReLU(True),
            nn.Conv2d(kenh_co_so*2, kenh_co_so*4, 3, stride=2, padding=1), nn.BatchNorm2d(kenh_co_so*4), nn.ReLU(True),
            nn.Conv2d(kenh_co_so*4, kenh_co_so*8, 3, stride=2, padding=1), nn.BatchNorm2d(kenh_co_so*8), nn.ReLU(True),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(kenh_co_so*8, kenh_co_so*4, 4, stride=2, padding=1), nn.BatchNorm2d(kenh_co_so*4), nn.ReLU(True),
            nn.ConvTranspose2d(kenh_co_so*4, kenh_co_so*2, 4, stride=2, padding=1), nn.BatchNorm2d(kenh_co_so*2), nn.ReLU(True),
            nn.ConvTranspose2d(kenh_co_so*2, kenh_co_so, 4, stride=2, padding=1), nn.BatchNorm2d(kenh_co_so), nn.ReLU(True),
            nn.ConvTranspose2d(kenh_co_so, so_kenh_vao, 4, stride=2, padding=1), nn.Sigmoid()
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.decoder(self.encoder(x))


class Discriminator(nn.Module):
    """Discriminator phân biệt ảnh thực vs giả bằng CNN."""
    def __init__(self, so_kenh_vao: int = 3, kenh_co_so: int = 32):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(so_kenh_vao, kenh_co_so, 4, stride=2, padding=1), nn.LeakyReLU(0.2, True),
            nn.Conv2d(kenh_co_so, kenh_co_so*2, 4, stride=2, padding=1), nn.BatchNorm2d(kenh_co_so*2), nn.LeakyReLU(0.2, True),
            nn.Conv2d(kenh_co_so*2, kenh_co_so*4, 4, stride=2, padding=1), nn.BatchNorm2d(kenh_co_so*4), nn.LeakyReLU(0.2, True),
            nn.Conv2d(kenh_co_so*4, kenh_co_so*8, 4, stride=2, padding=1), nn.BatchNorm2d(kenh_co_so*8), nn.LeakyReLU(0.2, True),
            nn.AdaptiveAvgPool2d((1, 1)),
        )
        self.fc = nn.Linear(kenh_co_so*8, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.net(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)


# Khởi tạo GAN
generator_gan = Generator(so_kenh_vao=3, kenh_co_so=so_kenh_co_so).to(thiet_bi)
discriminator_gan = Discriminator(so_kenh_vao=3, kenh_co_so=so_kenh_co_so).to(thiet_bi)

# ============ WRAP VỚI DataParallel NẾU CÓ NHIỀU GPU ============
if use_multi_gpu:
    print(f"\n🚀 Wrapping GAN với DataParallel ({n_gpus} GPUs)")
    generator_gan = nn.DataParallel(generator_gan)
    discriminator_gan = nn.DataParallel(discriminator_gan)
    print(f"✅ Generator device_ids: {generator_gan.device_ids}")
    print(f"✅ Discriminator device_ids: {discriminator_gan.device_ids}")

print("✅ Generator:")
print(generator_gan)
print("\n✅ Discriminator:")
print(discriminator_gan)

if use_multi_gpu:
    print(f"\n🎮 Multi-GPU GAN Training ENABLED: {n_gpus} GPUs đồng thời!")

In [ ]:
# ==================== HÀM HỖ TRỢ GAN (TỐI ƯU GPU + MULTI-GPU) ====================

def train_gan(generator: nn.Module, discriminator: nn.Module, 
              train_loader: DataLoader, val_loader: DataLoader,
              epochs: int, device: torch.device, lambda_recon: float = 100.0) -> tuple:
    """Huấn luyện GAN cho denoising với Adversarial Loss + Reconstruction Loss.
    
    Tối ưu hóa cho GPU:
    - Mixed Precision Training (FP16)
    - Multi-GPU support với DataParallel
    - GPU memory efficient training
    """
    
    optim_g = torch.optim.Adam(generator.parameters(), lr=2e-4, betas=(0.5, 0.999))
    optim_d = torch.optim.Adam(discriminator.parameters(), lr=2e-4, betas=(0.5, 0.999))
    
    loss_gan = nn.BCEWithLogitsLoss()
    loss_recon = nn.L1Loss()
    
    history = {'g_loss': [], 'd_loss': [], 'psnr_val': [], 'mse_val': [], 'best_psnr': 0}
    best_psnr = 0.0
    best_checkpoint_gan = None
    
    print("\n" + "="*60)
    print("🚀 BẮT ĐẦU HUẤN LUYỆN GAN")
    if use_multi_gpu:
        print(f"🎮 Training với {n_gpus} GPUs: {[torch.cuda.get_device_name(i) for i in range(n_gpus)]}")
    print("="*60)
    
    # Start timing
    time_start = time.time()
    
    for epoch in range(epochs):
        generator.train()
        discriminator.train()
        
        g_loss_epoch, d_loss_epoch = 0.0, 0.0
        so_batches = 0
        epoch_start = time.time()
        
        with tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False) as pbar:
            for batch_idx, (img_nhieu, _) in enumerate(pbar):
                img_nhieu = img_nhieu.to(device, non_blocking=True)  # Non-blocking transfer
                img_sach = img_nhieu.clone()
                B = img_nhieu.size(0)
                
                # ============ CẬP NHẬT DISCRIMINATOR ============
                optim_d.zero_grad()
                
                # Sử dụng autocast nếu mixed precision được bật
                if use_amp:
                    with autocast():
                        # Loss discriminator trên ảnh thật
                        label_thuc = torch.ones(B, 1, device=device)
                        output_thuc = discriminator(img_sach)
                        d_loss_thuc = loss_gan(output_thuc, label_thuc)
                        
                        # Loss discriminator trên ảnh giả
                        with torch.no_grad():
                            img_tao = generator(img_nhieu)
                        label_gia = torch.zeros(B, 1, device=device)
                        output_gia = discriminator(img_tao.detach())
                        d_loss_gia = loss_gan(output_gia, label_gia)
                        
                        d_loss = d_loss_thuc + d_loss_gia
                    
                    scaler.scale(d_loss).backward()
                    scaler.unscale_(optim_d)
                    torch.nn.utils.clip_grad_norm_(discriminator.parameters(), 1.0)
                    scaler.step(optim_d)
                    scaler.update()
                else:
                    # Loss discriminator trên ảnh thật - DataParallel tự động phân chia
                    label_thuc = torch.ones(B, 1, device=device)
                    output_thuc = discriminator(img_sach)
                    d_loss_thuc = loss_gan(output_thuc, label_thuc)
                    
                    # Loss discriminator trên ảnh giả
                    with torch.no_grad():
                        img_tao = generator(img_nhieu)
                    label_gia = torch.zeros(B, 1, device=device)
                    output_gia = discriminator(img_tao.detach())
                    d_loss_gia = loss_gan(output_gia, label_gia)
                    
                    d_loss = d_loss_thuc + d_loss_gia
                    d_loss.backward()
                    torch.nn.utils.clip_grad_norm_(discriminator.parameters(), 1.0)
                    optim_d.step()
                
                # ============ CẬP NHẬT GENERATOR ============
                optim_g.zero_grad()
                
                if use_amp:
                    with autocast():
                        img_tao = generator(img_nhieu)
                        
                        # Adversarial loss
                        output_gia = discriminator(img_tao)
                        g_loss_adv = loss_gan(output_gia, label_thuc)
                        
                        # Reconstruction loss
                        g_loss_recon = loss_recon(img_tao, img_sach)
                        
                        # Tổng loss
                        g_loss = g_loss_adv + lambda_recon * g_loss_recon
                    
                    scaler.scale(g_loss).backward()
                    scaler.unscale_(optim_g)
                    torch.nn.utils.clip_grad_norm_(generator.parameters(), 1.0)
                    scaler.step(optim_g)
                    scaler.update()
                else:
                    img_tao = generator(img_nhieu)
                    
                    # Adversarial loss
                    output_gia = discriminator(img_tao)
                    g_loss_adv = loss_gan(output_gia, label_thuc)
                    
                    # Reconstruction loss
                    g_loss_recon = loss_recon(img_tao, img_sach)
                    
                    # Tổng loss
                    g_loss = g_loss_adv + lambda_recon * g_loss_recon
                    g_loss.backward()
                    torch.nn.utils.clip_grad_norm_(generator.parameters(), 1.0)
                    optim_g.step()
                
                g_loss_epoch += g_loss.item()
                d_loss_epoch += d_loss.item()
                so_batches += 1
                
                pbar.set_postfix({'G': f'{g_loss.item():.3f}', 'D': f'{d_loss.item():.3f}'})
        
        # Trung bình loss
        g_loss_epoch /= so_batches
        d_loss_epoch /= so_batches
        history['g_loss'].append(g_loss_epoch)
        history['d_loss'].append(d_loss_epoch)
        
        # ============ ĐÁNH GIÁ VALIDATION ============
        generator.eval()
        psnr_vals, mse_vals = [], []
        
        with torch.no_grad():
            for img_nhieu, _ in val_loader:
                img_nhieu = img_nhieu.to(device, non_blocking=True)
                img_sach = img_nhieu.clone()
                img_tao = generator(img_nhieu)
                
                psnr_vals.append(psnr_tensor(img_tao, img_sach).item())
                mse_vals.append(mse_tensor(img_tao, img_sach).item())
        
        psnr_mean = np.mean(psnr_vals)
        mse_mean = np.mean(mse_vals)
        history['psnr_val'].append(psnr_mean)
        history['mse_val'].append(mse_mean)
        
        # Tính thời gian epoch
        epoch_time = time.time() - epoch_start
        
        # Lưu checkpoint tốt nhất
        if psnr_mean > best_psnr:
            best_psnr = psnr_mean
            
            # Lưu state_dict - nếu dùng DataParallel, lưu module.state_dict()
            if use_multi_gpu:
                best_checkpoint_gan = {
                    'generator': generator.module.state_dict(),
                    'discriminator': discriminator.module.state_dict(),
                    'epoch': epoch
                }
                torch.save(generator.module.state_dict(), duong_dan_checkpoint_gan_g)
                torch.save(discriminator.module.state_dict(), duong_dan_checkpoint_gan_d)
            else:
                best_checkpoint_gan = {
                    'generator': generator.state_dict(),
                    'discriminator': discriminator.state_dict(),
                    'epoch': epoch
                }
                torch.save(generator.state_dict(), duong_dan_checkpoint_gan_g)
                torch.save(discriminator.state_dict(), duong_dan_checkpoint_gan_d)
            
            history['best_psnr'] = best_psnr
            
            print(f"✅ Epoch {epoch+1} ({epoch_time:.1f}s): G={g_loss_epoch:.4f}, D={d_loss_epoch:.4f}, "
                  f"PSNR={psnr_mean:.3f}dB [SAVED]")
        else:
            print(f"   Epoch {epoch+1} ({epoch_time:.1f}s): G={g_loss_epoch:.4f}, D={d_loss_epoch:.4f}, "
                  f"PSNR={psnr_mean:.3f}dB")
    
    if best_checkpoint_gan:
        if use_multi_gpu:
            generator.module.load_state_dict(best_checkpoint_gan['generator'])
            discriminator.module.load_state_dict(best_checkpoint_gan['discriminator'])
        else:
            generator.load_state_dict(best_checkpoint_gan['generator'])
            discriminator.load_state_dict(best_checkpoint_gan['discriminator'])
    
    total_time = time.time() - time_start
    print("="*60)
    print(f"🎉 HOÀN THÀNH HỌC GAN")
    print(f"   Best PSNR: {best_psnr:.4f}")
    print(f"   Thời gian: {total_time/60:.1f} phút ({total_time/3600:.2f} giờ)")
    print(f"   Thiết bị: {device}")
    if use_multi_gpu:
        print(f"   🎮 Đã sử dụng {n_gpus} GPUs đồng thời!")
    print("="*60)
    
    return history, generator, discriminator

In [ ]:
# ==================== CHẠY HUẤN LUYỆN GAN ====================

# Gọi hàm train_gan để huấn luyện
history_gan, generator_gan, discriminator_gan = train_gan(
    generator_gan, 
    discriminator_gan, 
    train_loader, 
    val_loader, 
    epochs=so_epoch, 
    device=thiet_bi, 
    lambda_recon=100.0
)


In [ ]:
# ==================== ĐỒ THỊ SO SÁNH HỌC HUẤN LUYỆN ====================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('So sánh quá trình huấn luyện: Autoencoder vs GAN', fontsize=16, fontweight='bold')

# G Loss
axes[0, 0].plot(history_gan['g_loss'], 'b-', linewidth=2, label='Generator Loss')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Generator Loss qua các epoch')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].legend()

# D Loss
axes[0, 1].plot(history_gan['d_loss'], 'r-', linewidth=2, label='Discriminator Loss')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].set_title('Discriminator Loss qua các epoch')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].legend()

# PSNR Comparison
axes[1, 0].plot(lich_su['psnr_val'], 'g-o', linewidth=2, label='Autoencoder', markersize=5)
axes[1, 0].plot(history_gan['psnr_val'], 'b-s', linewidth=2, label='GAN', markersize=5)
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('PSNR (dB)')
axes[1, 0].set_title('PSNR trên validation set')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].legend()

# MSE Comparison
axes[1, 1].plot(lich_su['mse_val'], 'g-o', linewidth=2, label='Autoencoder', markersize=5)
axes[1, 1].plot(history_gan['mse_val'], 'b-s', linewidth=2, label='GAN', markersize=5)
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('MSE')
axes[1, 1].set_title('MSE trên validation set')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].legend()

plt.tight_layout()
plt.savefig(thu_muc_ket_qua / 'comparison_training.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Đã lưu biểu đồ so sánh huấn luyện")


In [ ]:
# ==================== LƯU ẢNH MINH HỌA TỪ MODEL GAN TỐT NHẤT ====================

print("\n" + "="*60)
print("🖼️ LƯU ẢNH MINH HỌA TỪ GAN TỐT NHẤT")
print("="*60)

# Lấy 1 batch từ validation set để minh hoạ
best_gan_sample, _ = next(iter(val_loader))
luu_minh_hoa(generator_gan, best_gan_sample, thiet_bi, loai_nhieu, do_manh_nhieu, thu_muc_ket_qua, tag="best_gan_generator")

print(f"✅ Đã lưu ảnh GAN chất lượng cao nhất (PSNR={history_gan['best_psnr']:.4f}dB)")
print("="*60)

# Clear cache
if use_cuda:
    torch.cuda.empty_cache()
    print("\n💾 GPU cache đã được xóa")


In [ ]:
# ==================== BẢNG SO SÁNH CHUYÊN SÂUÂU ====================

# Tính độ đo trên toàn bộ validation set
def evaluate_model(model: nn.Module, val_loader: DataLoader, device: torch.device) -> dict:
    """Đánh giá chi tiết mô hình trên validation set."""
    model.eval()
    psnr_vals, mse_vals, l1_vals = [], [], []
    
    with torch.no_grad():
        for img_clean, _ in val_loader:
            img_clean = img_clean.to(device)
            img_noisy = them_nhieu(img_clean, loai_nhieu, do_manh_nhieu).to(device, non_blocking=True)
            img_denoised = model(img_noisy)
            
            psnr_vals.append(psnr_tensor(img_denoised, img_clean).item())
            mse_vals.append(mse_tensor(img_denoised, img_clean).item())
            l1_vals.append(torch.nn.functional.l1_loss(img_denoised, img_clean).item())
    
    return {
        'psnr_mean': np.mean(psnr_vals),
        'psnr_std': np.std(psnr_vals),
        'mse_mean': np.mean(mse_vals),
        'mse_std': np.std(mse_vals),
        'l1_mean': np.mean(l1_vals),
        'l1_std': np.std(l1_vals),
    }

# Đánh giá Autoencoder
metrics_ae = evaluate_model(mo_hinh, val_loader, thiet_bi)

# Đánh giá GAN
metrics_gan = evaluate_model(generator_gan, val_loader, thiet_bi)

# Tạo bảng so sánh
print("\n" + "="*80)
print(" "*20 + "📊 BẢNG SO SÁNH AUTOENCODER vs GAN")
print("="*80)
print(f"{'Chỉ số':25} {'Autoencoder':^25} {'GAN':^25}")
print("-"*80)
print(f"{'PSNR (dB)':25} {metrics_ae['psnr_mean']:>10.4f} ± {metrics_ae['psnr_std']:>6.4f}   |   "
      f"{metrics_gan['psnr_mean']:>10.4f} ± {metrics_gan['psnr_std']:>6.4f}")
print(f"{'MSE':25} {metrics_ae['mse_mean']:>10.6f} ± {metrics_ae['mse_std']:>6.6f}   |   "
      f"{metrics_gan['mse_mean']:>10.6f} ± {metrics_gan['mse_std']:>6.6f}")
print(f"{'L1 Loss':25} {metrics_ae['l1_mean']:>10.6f} ± {metrics_ae['l1_std']:>6.6f}   |   "
      f"{metrics_gan['l1_mean']:>10.6f} ± {metrics_gan['l1_std']:>6.6f}")
print("="*80)

# Xác định mô hình tốt hơn
if metrics_gan['psnr_mean'] > metrics_ae['psnr_mean']:
    print(f"🏆 KẾT LUẬN: GAN vượt trội hơn với PSNR cao hơn {metrics_gan['psnr_mean'] - metrics_ae['psnr_mean']:.4f} dB")
else:
    print(f"🏆 KẾT LUẬN: Autoencoder vượt trội hơn với PSNR cao hơn {metrics_ae['psnr_mean'] - metrics_gan['psnr_mean']:.4f} dB")

print(f"\n📈 Tất cả chỉ số chi tiết:")
print(f"   Autoencoder: {metrics_ae}")
print(f"   GAN:         {metrics_gan}")


In [ ]:
# ==================== HÌNH ẢNH SO SÁNH TRỰC QUAN ====================

def visualize_comparison(val_loader: DataLoader, autoencoder: nn.Module, 
                        generator: nn.Module, device: torch.device, n_samples: int = 4):
    """So sánh trực quan kết quả Autoencoder vs GAN."""
    
    autoencoder.eval()
    generator.eval()
    
    fig, axes = plt.subplots(n_samples, 4, figsize=(16, 4*n_samples))
    fig.suptitle('So sánh kết quả Autoencoder vs GAN trên validation set', 
                 fontsize=14, fontweight='bold', y=0.995)
    
    sample_count = 0
    
    with torch.no_grad():
        for img_clean, _ in val_loader:
            if sample_count >= n_samples:
                break
            
            img_clean = img_clean.to(device)
            img_noisy = them_nhieu(img_clean, loai_nhieu, do_manh_nhieu).to(device, non_blocking=True)
            
            for i in range(img_clean.size(0)):
                if sample_count >= n_samples:
                    break
                
                # Forward pass
                ae_output = autoencoder(img_noisy[i:i+1])
                gan_output = generator(img_noisy[i:i+1])
                
                # Tính PSNR
                psnr_ae = psnr_tensor(ae_output, img_clean[i:i+1]).item()
                psnr_gan = psnr_tensor(gan_output, img_clean[i:i+1]).item()
                
                # Chuyển sang NumPy để hiển thị
                def to_display(tensor):
                    return tensor.squeeze(0).permute(1, 2, 0).cpu().numpy()
                
                img_noisy_np = to_display(img_noisy[i:i+1])
                img_clean_np = to_display(img_clean[i:i+1])
                ae_np = to_display(ae_output)
                gan_np = to_display(gan_output)
                
                # Hàng (i)
                row = sample_count
                
                # Cột 0: Ảnh nhiễu
                axes[row, 0].imshow(img_noisy_np)
                axes[row, 0].set_title(f'Ảnh nhiễu\n({loai_nhieu}, {do_manh_nhieu:.2f})', fontsize=10)
                axes[row, 0].axis('off')
                
                # Cột 1: Ảnh gốc
                axes[row, 1].imshow(img_clean_np)
                axes[row, 1].set_title('Ảnh gốc (Ground Truth)', fontsize=10)
                axes[row, 1].axis('off')
                
                # Cột 2: Kết quả Autoencoder
                axes[row, 2].imshow(ae_np)
                axes[row, 2].set_title(f'Autoencoder\nPSNR={psnr_ae:.2f}dB', 
                                      fontsize=10, fontweight='bold', color='green')
                axes[row, 2].axis('off')
                
                # Cột 3: Kết quả GAN
                axes[row, 3].imshow(gan_np)
                axes[row, 3].set_title(f'GAN\nPSNR={psnr_gan:.2f}dB', 
                                      fontsize=10, fontweight='bold', color='blue')
                axes[row, 3].axis('off')
                
                sample_count += 1
    
    plt.tight_layout()
    plt.savefig(thu_muc_ket_qua / 'ae_vs_gan_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"✅ Đã lưu hình ảnh so sánh (đã so sánh {sample_count} mẫu)")

# Hiển thị so sánh
visualize_comparison(val_loader, mo_hinh, generator_gan, thiet_bi, n_samples=4)
